## Set up google colab environment

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [ ]:
from SGCNN_7_Utils import *
import scipy.io as sio

## Load Indian Pines Dataset - Source

In [ ]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [ ]:
data_source = uIndianPines['indian_pines_corrected']
ground_truth_source = gt_IndianPines['indian_pines_gt']

In [ ]:
data_source.shape

(145, 145, 200)

In [ ]:
ground_truth_source.shape

(145, 145)

# Load target dataset

In [ ]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [ ]:
data_target = uBotswana['Botswana']
ground_truth_target = gt_Botswana['Botswana_gt']

In [ ]:
data_target.shape

(1476, 256, 145)

In [ ]:
ground_truth_target.shape

(1476, 256)

## Distrubution of samples for each class in Source

In [ ]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [ ]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_source

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

# Class distribution of samples in Target

In [ ]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [ ]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[1:] ## Dropping classes with small number of samples
classes_target

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with different overlap ratios and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [ ]:
pretrain_results = pretrain_source_models(percentages = [60,70,80,90],
                                          classes = classes_source,
                                          cube_size = 20,
                                          overlap_ratios = [0.8,0.9,1],
                                          data = data_source,
                                          ground_truth = ground_truth_source,
                                          batch_size = 20,
                                          channels = 64,
                                          epochs = 100,
                                          Verbosity = 1,
                                          accuracies = [],
                                          learning_rate = 0.0001,
                                          source_dataset = 'indian_pines')

Streaming output truncated to the last 5000 lines.

Epoch 00025: val_categorical_accuracy did not improve from 0.68917
Epoch 26/100
118/118 [==============================] - 2s 14ms/step - loss: 2.9846 - categorical_accuracy: 0.7045 - val_loss: 3.1074 - val_categorical_accuracy: 0.6720

Epoch 00026: val_categorical_accuracy did not improve from 0.68917
Epoch 27/100
118/118 [==============================] - 2s 16ms/step - loss: 2.9714 - categorical_accuracy: 0.7034 - val_loss: 3.1437 - val_categorical_accuracy: 0.6497

Epoch 00027: val_categorical_accuracy did not improve from 0.68917
Epoch 28/100
118/118 [==============================] - 2s 13ms/step - loss: 2.9060 - categorical_accuracy: 0.7262 - val_loss: 3.1142 - val_categorical_accuracy: 0.6637

Epoch 00028: val_categorical_accuracy did not improve from 0.68917
Epoch 29/100
118/118 [==============================] - 2s 15ms/step - loss: 2.9136 - categorical_accuracy: 0.7100 - val_loss: 3.2425 - val_categorical_accuracy: 0.6000



In [17]:
pretrain_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,477,323,60,59.13
1,0.8,555,245,70,62.04
2,0.8,636,164,80,68.90
3,0.8,715,85,90,72.94
4,0.9,2348,1570,60,76.43
5,0.9,2740,1178,70,78.78
6,0.9,3132,786,80,77.74
7,0.9,3523,395,90,78.48
8,1.0,4740,3166,60,75.17
9,1.0,5530,2376,70,74.66


# Fine tune on botswana

In [18]:
transfer_results, confusion_matrixes = transfer_learning(percentages = [60,70,80,90],
                                                        source_dataset = 'indian_pines',
                                                        target_dataset = 'botswana',
                                                        data = data_target,
                                                        ground_truth = ground_truth_target,
                                                        classes = classes_target,
                                                        overlap_ratios = [0.8,0.9,1],
                                                        channels = 64,
                                                        cube_size = 20,
                                                        learning_rate = 0.0001,
                                                        epochs = 100,
                                                        batch_size = 20)

Streaming output truncated to the last 5000 lines.




Model training starts for data with overlap ratio 0.8 and 90 percent samples from each class in training set 

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 20, 20, 64)] 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 20, 20, 64)   4160        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 20, 20, 64)   256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation

# Transfer Learning results on Botswana

In [19]:
transfer_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,251,175,60,68.57
1,0.8,293,133,70,80.45
2,0.8,337,89,80,86.52
3,0.8,378,48,90,75.00
4,0.9,914,615,60,82.60
5,0.9,1065,464,70,89.01
6,0.9,1219,310,80,90.97
7,0.9,1371,158,90,96.20
8,1.0,1880,1264,60,91.77
9,1.0,2195,949,70,84.72


# Classification accuracies per class for each model

In [20]:
for cm in confusion_matrixes:
  print(cm)

                1  2   3   4   5  ...  11  12  13  14  classfication_accuracies
0              12  0   0   0   2  ...   0   0   0   0                     85.71
1               0  0   0   0   6  ...   0   0   0   0                       0.0
2               0  0  13   0   0  ...   0   0   0   0                     92.86
3               0  0   0   0   0  ...   0   0   0   0                       0.0
4               3  0   0   0  17  ...   0   0   1   0                     70.83
5               0  0   0   0   0  ...   0   0  10   0                     28.57
6               4  0   0   0   0  ...   0   0   0   0                     69.23
7               0  0   0   0   0  ...   0   2   0   0                      80.0
8               0  0   0   0   1  ...   0   0   0   0                      75.0
9               0  0   0   0   0  ...   0   0   0   0                     100.0
10              0  0   0   0   0  ...  10   0   0   0                     76.92
11              0  0   0   0   0  ...   